Control Panel\Hardware and Sound\Devices and Printers

In [1]:
import serial
ser=serial.Serial("COM7",9600,timeout=1)
ser.write(bytes(f"{1},{1}\n", 'utf-8'))

4

In [2]:
import cv2 as cv
from cv2 import aruco
import numpy as np

In [3]:
calib_data_path = "./MultiMatrix.npz"
calib_data = np.load(calib_data_path)
print(calib_data.files)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
r_vectors = calib_data["rVector"]
t_vectors = calib_data["tVector"]

MARKER_SIZE = 8  # centimeters
marker_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
param_markers = aruco.DetectorParameters_create()

['camMatrix', 'distCoef', 'rVector', 'tVector']


In [ ]:
cap = cv.VideoCapture(1)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    marker_corners, marker_IDs, reject = aruco.detectMarkers(
        gray_frame, marker_dict, parameters=param_markers
    )
    if marker_corners:
        rVec, tVec, _ = aruco.estimatePoseSingleMarkers(
            marker_corners, MARKER_SIZE, cam_mat, dist_coef
        )
        total_markers = range(0, marker_IDs.size)
        for ids, corners, i in zip(marker_IDs, marker_corners, total_markers):
            cv.polylines(
                frame, [corners.astype(np.int32)], True, (0, 255, 255), 4, cv.LINE_AA
            )
            corners = corners.reshape(4, 2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()

            # Since there was mistake in calculating the distance approach point-outed in the Video Tutorial's comment
            # so I have rectified that mistake, I have test that out it increase the accuracy overall.
            # Calculating the distance
            distance = np.sqrt(
                tVec[i][0][2] ** 2 + tVec[i][0][0] ** 2 + tVec[i][0][1] ** 2
            )
            
            
            # Printing coordinate of distance
            coordinate = str(int(tVec[i][0][0]))+","+str(int(tVec[i][0][2]))
            ser.write(bytes(f"{int(tVec[i][0][0])},{int(tVec[i][0][2])}\n", 'utf-8'))
            print(coordinate)
            
            
            # Draw the pose of the marker
            point = cv.drawFrameAxes(frame, cam_mat, dist_coef, rVec[i], tVec[i], 4, 4)
            cv.putText(
                frame,
                f"id: {ids[0]} Dist: {round(distance, 2)}",
                top_right,
                cv.FONT_HERSHEY_PLAIN,
                1.3,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            cv.putText(
                frame,
                f"x:{round(tVec[i][0][0],1)} y: {round(tVec[i][0][2],1)} ",
                bottom_right,
                cv.FONT_HERSHEY_PLAIN,
                1.0,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            # print(ids, "  ", corners)
    cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord("q"):
        break
cap.release()
cv.destroyAllWindows()

-14,579
-14,561
-17,592
19,241
71,380
71,384
68,374
69,386
70,386
71,382
73,389
59,383
62,396
61,390
61,390
61,387
61,387
66,375
63,366
63,357
66,350
12,25
12,221
-4,177
-3,176
-6,165
-17,124
-14,116
-14,116
-14,115
-17,99
-17,99
-15,96
-15,91
-32,158
-32,158
-31,161
32,167
32,167
34,166
36,167
36,166
37,168
39,172
42,167
43,149
35,128
0,174
25,393
26,406
26,406
179,486
182,493
189,510
182,487
181,481
181,481
189,502
189,503
181,481
181,481
183,486
183,486
183,486
183,486
174,461
174,461
181,481
174,461
184,488
184,488
183,486
183,486
182,482
182,482
182,482
182,482
174,462
174,462
174,462
174,462
174,462
174,462
175,465
175,465
175,465
175,465
175,465
175,465
175,465
182,486
182,486
182,486
181,482
176,467
183,486
164,418
163,413
170,427
164,411
162,406
167,417
167,417
167,417
162,404
168,417
168,416
164,406
164,405
169,412
171,412
167,400
166,397
166,397
172,409
167,396
173,409
170,402
170,402
171,402
173,408
172,404
172,404
172,404
169,397
167,391
174,408
167,391
167,391
174,407
169

In [ ]:
cap.release()
cv.destroyAllWindows()